<a href="https://colab.research.google.com/github/Benjamin025/Benjamin025/blob/main/Forest_Change_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forest Change In the Aberdares and its impact on air quality based Nitrogen Dioxide Levels
===

Forests play a critical role in maintaining ecological balance and air quality. The Aberdare Forest which one of the main forests in Kenya is the focus of this analysis, the changes between 2018 and 2024 are examined in the subsequent sections. The analysis will also look at how the air quality in the perspective of atmospheric Nitrogen Dioxide levels in the area responds in relation to the recorded change.

The methodology used involves selecting Sentinel 2 Level 2A images at three year intervals from 2018 to 2024 i.e 2018 , 2021 and 2024. The images were taken around the same time which is between 17th February and 27th
February when the cloud cover is minimal.
The next step is to perform land cover classification to identify the forest extents, due to the complexity of collecting ground data to train models which may be time consuming especially for large
datasets we used the sen2classification library in python. The library is designed for automatic land cover classification of Sentinel 2 images using machine learning. The library computes vegetation indices such as
NDVI, BAEI, AWEI and NDTI for thresholding and uses either ALCC or gradient boosting. The output classes are as follows;
            0 = no data
            1 = water
            2 = low vegetation
            3 = high vegetation
            4 = soil
            5 = built up
For more information visit the __[PyPi website](https://pypi.org/project/sen2classification/)__

Due to space requirements of the algorithm running the algorithm in Dunia was not possible, it had to be done on the local computer environment. This __[notebook](https://github.com/ivybart/GMES_Forest_Change)__ shows how to use the sen2classification library is used to
obtain the classified images. For those with enough space running the same code in the Dunia platform works too. Let's get started!!

## Forest Change Analysis

The 'rasterio' package in Python is designed for reading, writing, and manipulating raster data.
The 'numPy' for Numerical Python widely used for scientific computing.
The 'matplolib' is used for plotting static, animated, and interactive visualizations
The 'os' module is operating system used to handle directory and file operations, such as reading, writing, and    modifying     file attributes, along with fetching environment information and managing processes.
The 'ListedColormap' is a tool for creating colormaps from a list of specified colors.
The 'matplotlib.patches' module in Python is a Matplotlib library used to provides classes for drawing various shapes and       patches on Matplotlib figures.


In [ ]:
# Install the rasterstats package to be used in doing pixel count and area calculations
!pip install rasterstats

In [ ]:
# import required packages
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import geopandas as gpd
from skimage.measure import label, regionprops
from skimage.color import label2rgb

Create a list of imaged irectories that holds the paths for input and output directories for each image to be used in the processing.

In [ ]:
#a list for the input and output directories
image_directories = [
    ("data_raw/input", "data_raw/output"),
    ("data_raw/input", "data_raw/output"),
    ("data_raw/input", "data_raw/output"),
]


In [ ]:

def create_forest_binary_image(input_tiff, output_tiff):
    # Open the input TIFF file
    with rasterio.open(input_tiff) as src:
        # Read the data from the TIFF file
        data = src.read(1)  # Read the first band (assuming it's a single-band raster)

        # Create a binary image: 1 for forest (class ID 3), 0 for non-forest (all other class IDs)
        binary_forest = np.where(data == 3, 1, 0)

        # Define the metadata for the output file
        meta = src.meta.copy()
        meta.update(dtype=rasterio.uint8, count=1)

        # Write the binary image to the output TIFF file
        with rasterio.open(output_tiff, 'w', **meta) as dst:
            dst.write(binary_forest.astype(rasterio.uint8), 1)

    print(f"Binary forest image saved to {output_tiff}")

def process_images(years, input_dir, output_dir):
    for year in years:
        input_tiff = os.path.join(input_dir, f'{year}_LULC.tif')
        output_tiff = os.path.join(output_dir, f'{year}_binary_forest.tif')

        if os.path.exists(input_tiff):
            create_forest_binary_image(input_tiff, output_tiff)
        else:
            print(f"Input file for year {year} not found: {input_tiff}")

# Define the years you want to process
years = [2018, 2021, 2024]

# Define the input directory containing the TIFF files
input_dir = 'data_raw/input'

# Define the output directory to save the binary images
output_dir = 'data_raw/output'
os.makedirs(output_dir, exist_ok=True)

# Process the images
process_images(years, input_dir, output_dir)

The following code creates a binary image to classify the image into forest and non-forest,assigning forest as value 1 and all the other classes a value of 0.

In [ ]:
def display_binary_images(image_files, titles):
    # Define the colors for forest (green) and non-forest (cream)
    cmap = ListedColormap(['#FFFDD0', '#00FF00'])  # Cream and Green

    # Create a subplot grid
    fig, axs = plt.subplots(1, len(image_files), figsize=(15, 5))

    for i, file_path in enumerate(image_files):
        # Open the binary image file
        with rasterio.open(file_path) as src:
            # Read the data from the TIFF file
            binary_image = src.read(1)

        # Display the image
        ax = axs[i]
        im = ax.imshow(binary_image, cmap=cmap)
        ax.set_title(titles[i])


        # Add a legend for clarity
        legend_labels = [Patch(facecolor='#FFFDD0', label='Non-Forest'),
                         Patch(facecolor='#00FF00', label='Forest')]
        ax.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(1, 1), title='Legend')

    plt.suptitle('Forest (Green) and Non-Forest (Cream)')
    plt.tight_layout()
    plt.show()

# Define the paths to your binary image files
binary_image_files = [
    'data_raw/output/2018_binary_forest.tif',
    'data_raw/output/2021_binary_forest.tif',
    'data_raw/output/2024_binary_forest.tif'
]

# Define titles for each image
titles = ['2018 Forest Cover', '2021 Forest Cover', '2024 Forest Cover']

# Display the images in a grid with titles and legends
display_binary_images(binary_image_files, titles)

Compute the difference of the images to determine forest loss and gain

In [ ]:
def calculate_difference(image1_path, image2_path, output_path):
    # Open the first image
    with rasterio.open(image1_path) as src1:
        image1 = src1.read(1)

    # Open the second image
    with rasterio.open(image2_path) as src2:
        image2 = src2.read(1)

    # Calculate the difference
    difference = image2 - image1

    # Save the difference image
    meta = src1.meta.copy()
    meta.update(dtype=rasterio.int8, count=1)

    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(difference.astype(rasterio.int8), 1)

    print(f"Difference image saved to {output_path}")
    return difference

def display_difference_images(image_files, titles):
    # Define the colors for the difference image (negative: red, zero: cream, positive: green)
    cmap = ListedColormap(['#FF0000', '#FFFDD0', '#00FF00'])  # Red, Cream, Green

    # Create a subplot grid
    fig, axs = plt.subplots(1, len(image_files), figsize=(15, 5))

    for i, file_path in enumerate(image_files):
        # Open the difference image file
        with rasterio.open(file_path) as src:
            # Read the data from the TIFF file
            difference_image = src.read(1)

        # Display the image
        ax = axs[i]
        im = ax.imshow(difference_image, cmap=cmap, vmin=-1, vmax=1)
        ax.set_title(titles[i])
        ax.axis('on')


        # Add a legend for clarity
        legend_labels = [Patch(facecolor='#FF0000', label='Loss'),
                         Patch(facecolor='#FFFDD0', label='No Change'),
                         Patch(facecolor='#00FF00', label='Gain')]
        ax.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(1, 1), title='Legend')

    plt.suptitle('Difference in Land Cover: Loss (Red), No Change (Cream), Gain (Green)')
    plt.tight_layout()
    plt.show()

# Define the paths to your binary image files
image_2018 = 'data_raw/output/2018_binary_forest.tif'
image_2021 = 'data_raw/output/2021_binary_forest.tif'
image_2024 = 'data_raw/output/2024_binary_forest.tif'

# Define the output paths for the difference images
diff_2021_2018 = 'data_raw/output/diff_2021_2018.tiff'
diff_2024_2021 = 'data_raw/output/diff_2024_2021.tiff'

# Calculate the differences
difference_2021_2018 = calculate_difference(image_2018, image_2021, diff_2021_2018)
difference_2024_2021 = calculate_difference(image_2021, image_2024, diff_2024_2021)

# Define the paths to the difference image files and their titles
difference_image_files = [diff_2021_2018, diff_2024_2021]
titles = ['Difference 2021 - 2018', 'Difference 2024 - 2021']

# Display the difference images
display_difference_images(difference_image_files, titles)

### Calculating the zonal statistics using the binary images

In [ ]:
def calculate_pixel_statistics(difference_image, pixel_area_hectares):
    loss_pixels = np.sum(difference_image == -1)  # Pixels representing forest loss
    gain_pixels = np.sum(difference_image == 1)   # Pixels representing forest gain

    loss_area_ha = loss_pixels * pixel_area_hectares
    gain_area_ha = gain_pixels * pixel_area_hectares

    return {
        "loss_pixels": loss_pixels,
        "gain_pixels": gain_pixels,
        "loss_area_ha": loss_area_ha,
        "gain_area_ha": gain_area_ha
    }

# Define the paths to your difference image files
diff_2021_2018 = 'data_raw/output/diff_2021_2018.tiff'
diff_2024_2021 = 'data_raw/output/diff_2024_2021.tiff'

# Define the area of one pixel in hectares (assuming the pixel size is 10m x 10m)
pixel_area_hectares = (10 * 10) / 10000  # 1 hectare = 10,000 square meters

# Calculate and print pixel statistics for each difference image
with rasterio.open(diff_2021_2018) as src:
    difference_image_2021_2018 = src.read(1)
stats_2021_2018 = calculate_pixel_statistics(difference_image_2021_2018, pixel_area_hectares)

with rasterio.open(diff_2024_2021) as src:
    difference_image_2024_2021 = src.read(1)
stats_2024_2021 = calculate_pixel_statistics(difference_image_2024_2021, pixel_area_hectares)

print("Pixel statistics for 2021 - 2018 difference image:")
print(stats_2021_2018)

print("Pixel statistics for 2024 - 2021 difference image:")
print(stats_2024_2021)

# Prepare data for the bar graph
labels = ['2018-2021', '2021-2024']
loss_areas = [stats_2021_2018["loss_area_ha"], stats_2024_2021["loss_area_ha"]]
gain_areas = [stats_2021_2018["gain_area_ha"], stats_2024_2021["gain_area_ha"]]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
bars1 = ax.bar(x - width/2, loss_areas, width, label='Loss (ha)', color='red')
bars2 = ax.bar(x + width/2, gain_areas, width, label='Gain (ha)', color='green')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Time Period')
ax.set_ylabel('Area (hectares)')
ax.set_title('Forest Cover Change (Loss and Gain) by Time Period')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Attach a text label above each bar in bars1 and bars2, displaying its height.
def autolabel(bars):
    """Attach a text label above each bar in bars, displaying its height."""
    for bar in bars:
        height = bar.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(bars1)
autolabel(bars2)

fig.tight_layout()

plt.show()

## Air Quality Analysis

The air quality analysis was done by obtaining the images from Google Earth Engine , this is because the Sentinel 5P data comes in netcdf format which can be difficult to navigate. GEE also allows one to compute mean images from the image collection allowing one to get a complete image for their area of interest. The images were therefore prepared using GEE and uploaded into the dunia platform this __[Google Earth Engine Script](https://code.earthengine.google.com/9ca615dd59e03c498827a33021f66f3b)__ shows how you can also do the same for your area of interest.

The NO2 data is given in mols per square meter

In [ ]:
def plot_no2_images_horizontally(years, input_dir):
    # Setup the plot - determine the number of subplots based on the number of years
    fig, axs = plt.subplots(1, len(years), figsize=(15, 5))  # Adjust figsize as needed

    for idx, year in enumerate(years):
        input_tiff = os.path.join(input_dir, f'S5P_NO2_{year}.tif')

        if os.path.exists(input_tiff):
            try:
                # Open the TIFF file using rasterio
                with rasterio.open(input_tiff) as src:
                    # Read the data from the first band
                    data = src.read(1)

                    # Plot the image in the respective subplot
                    ax = axs[idx] if len(years) > 1 else axs
                    im = ax.imshow(data, cmap='viridis')
                    ax.set_title(f'{year} NO2 Image')
                    ax.set_xlabel('Pixel X Coordinates')
                    ax.set_ylabel('Pixel Y Coordinates')
                    ax.set_axis_off()  # Turn off axis numbering and ticks if preferred

            except Exception as e:
                print(f"Failed to read or plot {input_tiff}: {e}")
        else:
            print(f"Input file for year {year} not found: {input_tiff}")

    # Add a colorbar to the figure
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # x, y, width, height
    fig.colorbar(im, cax=cbar_ax)
    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust the right margin to accommodate the colorbar

    plt.show()

# Define the years you want to process
years = [2018, 2021, 2024]

# Define the input directory containing the TIFF files
input_dir = 'data_raw/input'

# Plot the NO2 images horizontally
plot_no2_images_horizontally(years, input_dir)


In [ ]:

def plot_difference(input_dir, years):
    data_arrays = []
    titles = []

    # Load and store data for specified years
    for year in years:
        input_tiff = os.path.join(input_dir, f'S5P_NO2_{year}.tif')

        if os.path.exists(input_tiff):
            with rasterio.open(input_tiff) as src:
                # Append data to the list after reading from the file
                data_arrays.append(src.read(1))
                titles.append(f'{year} NO2 Image')
        else:
            print(f"Input file for year {year} not found: {input_tiff}")
            return  # Exit if any file is missing

    # Compute differences between consecutive years
    differences = [data_arrays[i+1] - data_arrays[i] for i in range(len(data_arrays)-1)]
    titles_diff = [f'Difference {years[i+1]} - {years[i]}' for i in range(len(years)-1)]

    # Plotting
    fig, axs = plt.subplots(1, len(years) + len(differences), figsize=(20, 6))

    # Plot original images
    for idx, data in enumerate(data_arrays):
        ax = axs[idx]
        im = ax.imshow(data, cmap='viridis', vmin=np.percentile(data, 5), vmax=np.percentile(data, 95))
        ax.set_title(titles[idx])
        ax.set_axis_off()

    # Plot differences
    for idx, diff in enumerate(differences):
        ax = axs[len(years) + idx]
        im_diff = ax.imshow(diff, cmap='coolwarm', vmin=np.percentile(diff, 5), vmax=np.percentile(diff, 95))
        ax.set_title(titles_diff[idx])
        ax.set_axis_off()

    # Add colorbars
    fig.colorbar(im, ax=axs[:len(years)], orientation='vertical', fraction=0.02, pad=0.04)
    fig.colorbar(im_diff, ax=axs[len(years):], orientation='vertical', fraction=0.02, pad=0.04)

    plt.tight_layout()
    plt.show()

# Define the years you want to analyze
years = [2018, 2021, 2024]

# Define the input directory containing the TIFF files
input_dir = 'data_raw/input'

# Perform the analysis and plot the differences
plot_difference(input_dir, years)


In [ ]:


def calculate_and_plot_difference(input_dir, years):
    # Check and store data for each year
    images = {}
    for year in years:
        input_tiff = os.path.join(input_dir, f'S5P_NO2_{year}.tif')
        if os.path.exists(input_tiff):
            with rasterio.open(input_tiff) as src:
                images[year] = src.read(1)
        else:
            print(f"Input file for year {year} not found: {input_tiff}")
            return  # Exit if any file is missing

    # Compute and plot differences between consecutive years
    for i in range(len(years) - 1):
        year1, year2 = years[i], years[i+1]
        if year1 in images and year2 in images:
            difference = images[year2] - images[year1]

            # Plotting the difference
            plt.figure(figsize=(8, 6))
            plt.imshow(difference, cmap='coolwarm', vmin=np.percentile(difference, 5), vmax=np.percentile(difference, 95))
            plt.colorbar(label='Difference in NO2 Concentration')
            plt.title(f'Difference in NO2 Concentration: {year2} - {year1}')
            plt.xlabel('Pixel X Coordinates')
            plt.ylabel('Pixel Y Coordinates')
            plt.show()

# Define the years you want to analyze
years = [2018, 2021, 2024]

# Define the input directory containing the TIFF files
input_dir = 'data_raw/input'

# Calculate the differences and plot them
calculate_and_plot_difference(input_dir, years)


In [ ]:
# Define the paths to your raster files
raster1_path = 'data_raw/input/S5P_NO2_2018.tif'
raster2_path = 'data_raw/input/S5P_NO2_2021.tif'

# Open the first raster file
with rasterio.open(raster1_path) as src1:
    raster1 = src1.read(1)  # Read the first band

# Open the second raster file
with rasterio.open(raster2_path) as src2:
    raster2 = src2.read(1)  # Read the first band

# Compute the difference
difference1 = raster2 - raster1

# Print the result (or you could save it to a new file, if needed)
print(difference1)

In [ ]:
 # Plotting the difference
plt.figure(figsize=(8, 6))
plt.imshow(difference1, cmap='coolwarm', vmin=np.percentile(difference1, 5), vmax=np.percentile(difference1, 95))
plt.colorbar(label='Difference in NO2 Concentration')
plt.title(f'Difference in NO2 Concentration: {2021} - {2018}')
plt.xlabel('Pixel X Coordinates')
plt.ylabel('Pixel Y Coordinates')
plt.show()


In [ ]:
def calculate_and_plot_difference(image1_path, image2_path):
    # Load the first image
    with rasterio.open(image1_path) as src1:
        image1 = src1.read(1)

    # Load the second image
    with rasterio.open(image2_path) as src2:
        image2 = src2.read(1)

    # Calculate the difference
    difference = image2 - image1

    # Plotting the difference
    plt.figure(figsize=(8, 6))
    im = plt.imshow(difference, cmap='coolwarm', vmin=np.percentile(difference, 5), vmax=np.percentile(difference, 95))
    plt.colorbar(im, label='Difference in NO2 Concentration')
    plt.title('Difference in NO2 Concentration')
    plt.xlabel('Pixel X Coordinates')
    plt.ylabel('Pixel Y Coordinates')
    plt.show()

# Define the paths to the NO2 image files
image1_path = 'data_raw/input/S5P_NO2_2018.tif'
image2_path = 'data_raw/input/S5P_NO2_2021.tif'

# Calculate the differences and plot them
calculate_and_plot_difference(image1_path, image2_path)
